In [15]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
import mecab
from tqdm import tqdm

In [3]:
%ls

Mecab-ko-for-Google-Colab/ ctm.ipynb


In [7]:
import os
import json

In [6]:
text_file = '../preprocessing/data/result/np_data.json'

In [10]:
with open(text_file, 'r') as f:
    tmp = json.load(f)
contents = []
for c in tmp['data']:
    contents.append(c['text'])

In [12]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [16]:
custom_tokenizer = CustomTokenizer(mecab.MeCab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
train_bow_embeddings = vectorizer.fit_transform(contents)

/Users/elplaguister/opt/anaconda3/envs/proj_skt_corpus/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [18]:
print(train_bow_embeddings.shape)

(10753, 3000)


In [20]:
vocab = vectorizer.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}
len(vocab)

3000

In [21]:
train_ebds = bert_embeddings_from_list(contents,  "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

In [22]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_ebds, train_bow_embeddings, id2token)

In [23]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

0it [00:00, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 50])